# Part Ⅳ: Decompounded Options

### Import packages

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.integrate import quad
import matplotlib.pylab as plt
from math import log, sqrt, exp
import warnings
warnings.filterwarnings("ignore")

 # <a id = "top">Contents</a>

# [1. Define Functions and Models](#p1)
# [2. CMS Rate](#p2)
# [3. CMS Caplet](#p3)

#  <a id = "p1"> 1.<font color = "green">Define Functions and Models [(back to contents)](#top)

### A decompounded option pays the following at time T = 5y:
### <center color = black>$$\text{CMS}  10Y^{1/p} - 0.04^{1/q}$$
where p = 4 and q = 2. Use static replication to value the PV of this payoff.

For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
      h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
      h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

We can implement this in Python. First we define the IRR functions.

Let $m$ denote the payment frequenc ($m=2$ for semi-annual payment frequency), and let $N = T_N-T_n$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\mbox{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\mbox{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\mbox{IRR}'(K)&=-\frac{1}{K}\mbox{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\mbox{IRR}''(K)&=-\frac{2}{K}\mbox{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.


In [2]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative


#### Adjust the formula of g(F)

For CMS rate payment, since $g(F)=F$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &= K^{1/4} - 0.04^{1/2} \\
g'(K) &= 1/4*K^{-3/4} \\
g''(K) &= -3/16*K^{-7/4}
\end{split}
\end{equation*}

In [3]:
def g_0(K):
    return K**(1/4)-0.2

def g_1(K):
    return (1/4)*K**(-3/4)

def g_2(K):
    return (-3/16)*K**(-7/4)

The function $h(K) = g(K)/IRR(K)$ now simplies into:  
  
  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\mbox{IRR}(K)} \\
      h'(K) &= \frac{\mbox{IRR}(K)g'(K) - g(K)\mbox{IRR}'(K)}{\mbox{IRR}(K)^2} \\
      h''(K) &= \frac{\mbox{IRR}(K)g''(K)-\mbox{IRR}''(K)g(K) -2\cdot\mbox{IRR}'(K)g'(K)}{\mbox{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\mbox{IRR}'(K)^2g(K)}{\mbox{IRR}(K)^3}.
    \end{split}
  \end{equation*}


In [4]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

In [5]:
def BlackScholes76Call(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
def BlackScholes76Put(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    if F == K:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom
    return sabrsigma

In [6]:
def integral_rec(x,N,m,F,disc,sigma,T):
    h = h_2(x, m, N)
    Vrec=BlackScholes76Put(F, x, disc, sigma, T)
    return h*Vrec

def integral_pay(x,N,m,F,disc,sigma,T): 
    h = h_2(x, m, N)
    Vpay=BlackScholes76Call(F, x, disc, sigma, T)
    return h*Vpay

We will also need to implement the IRR-settled payer and receiver swaption formulae:

  \begin{equation*}
    \begin{split}
      V^{pay}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Call}(S_{n,N}(0),K,\sigma_{n,N},T) \\
      V^{rec}_{n,N}(0) &= D(0,T_n) \cdot \mbox{IRR}(S_{n,N}(0)) \cdot \mbox{Black76Put}(S_{n,N}(0),K,\sigma_{n,N},T) \\
    \end{split}
  \end{equation*}

where $S_{n,N}(0)=F$ is today's forward swap rate calculated based on the curves we bootstrapped, and $\sigma_{n,N}$ is the SABR implied volatility calibrated to swaption market data.

#  <a id = "p2"> 2.<font color = "green">CMS Rate [(back to contents)](#top)

\begin{equation*}
    \begin{split}
      D(0,T) g(F) + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

In [7]:
IRS = pd.read_excel("IRS_data_final.xlsx")
FSR = pd.read_excel("Forward Swap Rate.xlsx")
df_sigma = pd.read_csv('df_sigma.csv')

In [8]:
p = 4
q = 2

# F is CMS 5*10Y
# DF is discount factor

F = FSR.iloc[9]['Forward Swap Rate']
DF = IRS.iloc[9]['ON_DF']
sigmasabr = df_sigma.loc[9, 'sigma']

In [9]:
Alpha = pd.read_excel('Part2_SABR.xlsx',sheet_name = 'Alpha')
Rho = pd.read_excel('Part2_SABR.xlsx',sheet_name = 'Rho')
Nu = pd.read_excel('Part2_SABR.xlsx',sheet_name = 'Nu')

In [10]:
alpha = Alpha.iloc[1].iloc[4]
rho = Rho.iloc[1].iloc[4]
nu = Nu.iloc[1].iloc[4]

In [11]:
term2 = quad(lambda x: integral_rec(x, 10, 2, F, 0, SABR(F, x, 5, alpha, 0.9, rho, nu), 5), 0, F)
term3 = quad(lambda x: integral_pay(x, 10, 2, F, 0, SABR(F, x, 5, alpha, 0.9, rho, nu), 5), F, 1e3)
PV_1 = g_0(F) * DF + np.sum(term2 + term3)
print('The CMS Rate payment is: ', round(PV_1,4))

The CMS Rate payment is:  0.2497


#  <a id = "p3"> 3.<font color = "green"> CMS Caplet [(back to contents)](#top)

\begin{equation*}
    \begin{split}
      \ h'(L) V^{pay}(L)+ \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

where:
\begin{align*}
F^{\frac{1}{4}} &> 0.2 \\
F &> 0.2^4 \\
F &> 0.0016 = L
\end{align*}

In [12]:
L = 0.0016
term_1 = h_1(L, 2, 10) * BlackScholes76Call(F, L, 0, SABR(F, L, 5, alpha, 0.9, rho, nu),5)
term_2 = quad(lambda x: h_2(x, 2, 10) * BlackScholes76Call(F, x, 0, SABR(F, x, 5, alpha, 0.9, rho, nu),5), L, 1e3)
caplet = term_1 + term_2[0]
print('The CMS Caplet payment is: ', round(caplet,4))

The CMS Caplet payment is:  0.0349
